# rule-based机器人

本节的代码内容非常的简单，就是给大家展示一下rule-based的玩法，以及几个角度的升级。

## 版本1：最基础版本的rule-based机器人
基本上就是小学生级别的，问什么，答什么。

In [1]:
import random

# 打招呼
greetings = ['hola', 'hello', 'hi', 'Hi', 'Hey!', 'hey']
# 回复打招呼
random_greeting = random.choice(greetings)

# 对于“你怎么样？”这个问题的回复
question = ['How are you?', 'How are you doing?']
responses = ['Okay', "I'm fine"]
#随机选择一个回复
random_response = random.choice(responses)

# 机器人跑起来
while True:
    userInput = input('>>> ')
    if userInput in greetings:
        print(random_greeting)
    elif userInput in question:
        print(random_response)
    elif userInput == 'bye':
        break
    else:
        print('I did not uderstand what you said')

>>> 
I did not uderstand what you said
>>> hi
hello
>>> how are you
I did not uderstand what you said
>>> How are you?
Okay
>>> bye


## 升级I: intents意图
显然，这样的rule太弱智了，我们需要更好一点的“精准对答”。

比如：透过**关键词**来判断这句话的意图是什么？（intents）

In [2]:
from nltk import word_tokenize
import random

# 打招呼
greetings = ['hola', 'hello', 'hi', 'Hi', 'hey!','hey']
# 回复打招呼
random_greeting = random.choice(greetings)

# 对于“假期”的话题关键词
question = ['break','holiday','vacation','weekend']
# 回复假期话题
responses = ['It was nice! I went to Paris',"Sadly, I just stayed at home"]
# 随机选一个回
random_response = random.choice(responses)

# 机器人跑起来
while True:
    userInput = input('>>> ')
    # 验证用户输入，看看都有哪些词，提取关键词
    cleaned_input = word_tokenize(userInput)
    
    if not set(cleaned_input).isdisjoint(greetings): # set.isdisjoint 判断是否有交集，如果有则返回False，如无，返回True
        print(random_greeting)
    elif not set(cleaned_input).isdisjoint(question):
        print(random_response)
    elif userInput == 'bye':
        break
    else:
        print('I did not understand what you said.')

>>> hi
hey!
>>> how is your holiday?
Sadly, I just stayed at home
>>> bye


大家仍能发现，这依旧是文字层面的“精准对应”。 

现在主流的研究方向，是做到语义层面的对应。

比如：
- “肚子好饿哦”，“饭点到了”
都应该表示的是“要吃饭了”的意思。

在语义层面的对应上，就需要用到word vector之类的embedding方法。

## 升级II: 知识体系
有了知识体系，才能解决用户的问题。

我们可以用各种数据库，建立起一套体系，然后通过搜索的方式，来查找答案。

比如：   
最简单的就是用Python自己的graph数据结构来搭建一个“地图”，依据这个地图，我们可以清楚地找寻从一个地方到另一个地方的路径，然后作为回答， 反馈给用户。

In [3]:
# 建立一个机遇目标行业的database
# 比如：这里我们用的Python自带的graph

graph = {'上海': ['苏州', '常州'],
         '苏州': ['常州', '镇江'],
         '常州': ['镇江'],
         '镇江': ['常州'],
         '盐城': ['南通'],
         '南通': ['常州']}

# 明确如何找到从A到B的路径
def find_path(start, end, path=[]):
    path = path + [start]
    if start == end:
        return path
    if start not in graph:
        return None
    for node in graph[start]:
        if node not in path:
            newpath = find_path(node, end, path)
            if newpath:
                return newpath
    return None

In [4]:
print(find_path('上海','镇江'))

['上海', '苏州', '常州', '镇江']


同样的构建**知识图谱**的玩法，也可以使用一些Logic Programming，比如上个世纪学AI的同学都会学的Prolog，或者比如：python版本的prolog：PyKE。

他们可以构建一种复杂的逻辑网络，让你方便提取信息，而不至于需要你亲手code所有的信息：

son_of(bruce, thomas, norma)   
son_of(fred_a, thomas, norma)   
son_of(tim, thomas, norma)  
daughter_of(vicki, thomas, norma)  
daughter_of(jill, thomas, norma)    


## 升级III: 设计前端
任何行业，都分个前端和后端。
AI也不例外，我们这里讲的算法，都是在后端跑的。
那么，为了做一个靠谱的前端，很多项目往往也需要一个简单易用，靠谱的前端。

这里，利用Google的API，写一个类似钢铁侠Tony的语音小秘书Jarvis:

### 下面先来看一个最简单的说话版本
利用gTTS(Google Text-to-Speech API), 把文本转化为音频。

In [ ]:
from gtts import gTTS
import os 
tts = gTTS(text='你好，我是您的私人助手，我叫小辣椒', lang='zh-tw')
tts.save('hello.mp3')
os.system('mpg321 hello.mp3')

百度的助手API

科大讯飞的文字转语音的API

同理，有了文本到语音的功能，我们还可以运用Google API读出的Jarvis的回复：
（注意：这里需要你的机器安装几个库 SpeechRecognition, PyAudio和PySpeech）

pip install SpeechRecognition   
On OS X, install PortAudio using Homebrew: 
> brew install portaudio. 

Then, install PyAudio using Pip: 
> pip install pyaudio

In [6]:
import speech_recognition as sr
from time import ctime
import time
import os
from gtts import gTTS
import sys

In [7]:
# AI讲出来的话
def speak(audioString):
    print(audioString)
    tts = gTTS(text=audioString, lang='en')
    tts.save('audio.mp3')
    os.system('mpg321 audio.mp3')

In [8]:
# 录下来你讲的话
def recordAudio():
    # 用麦克风记录下你的话
    r = sr.Recognizer()
    with sr.Microphone() as source:
        audio = r.listen(source)
    
    # 用Google API转化音频
    data = ""
    try:
        data = r.recognize_goole(audio)
        print('You said:'+ data)
    except sr.UnknownValueError:
        print('Google Speech Recognition could not understand audio')
    except sr.RequestError as e:
        print('Could not request results from Google Speech Recognition service;{0}'.format(e))
        
    return data

In [10]:
# 自带的对话技能（rules）
def jarvis():
    while True:
        data = recordAudio()
        if "how are you" in data:
            speak('I am fine')
        if "what time is it" in data:
            speak(ctime())
        if "where is" in data:
            data = data.split(" ")
            location = data[2]
            speak('Hold on Tony, I will show you where' + location + 'is.')
            os.system('open -a Safari https://www.google.com/maps/place/' + location +'/&amp;')
            
        if "bye" in data:
            speak('bye bye')
            break
    

In [11]:
# 初始化
time.sleep(2)
speak('Hi Tony, what can I do for you?')

# 启动
jarvis()

Hi Tony, what can I do for you?


gTTSError: Connection error during token calculation: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x1114f42b0>: Failed to establish a new connection: [Errno 60] Operation timed out',))

不仅仅是语音前端。  

包括应用场景：微信、slack、Facebook Messager，等等，都可以把我们的ChatBot给Integrate进去。